## Lưu ý: Tạo file `raw_data.xlsx` với sheet = `country` trước khi chạy code

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from datetime import date,timedelta
import time

In [2]:
#Trả về DataFrame
def get_data():
    url = f"https://www.worldometers.info/coronavirus/"
    browser = webdriver.Chrome(executable_path="Driver/chromedriver.exe")
    browser.get(url)
    #
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    tbody = tree.find("tbody")
    trows = tbody.find_all("tr",{"role": "row"})
    total_row_world = tbody.find_all("tr",{"class" : "total_row_world"})
    list_data_country = [trow.text.split("\n")[1:-2] for trow in trows if trow not in total_row_world]
    #
    df_country = pd.DataFrame(list_data_country, columns = ["#", "Country, Other", "Total Cases", "New Cases", "Total Deaths", 
                                        "New Death", "Total Recovered", "Active Case", "Serious Critical", 
                                        "Tot Cases/1M pop", "Death/1M pop","Total test","Test/1M pop", "Population"])
    df_country.set_index("#", inplace=True)
    df_country["Time"] = date.today() - timedelta(1)
    return df_country

In [3]:
df_country = get_data()
df_country.head()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Death,Total Recovered,Active Case,Serious Critical,Tot Cases/1M pop,Death/1M pop,Total test,Test/1M pop,Population,Time
#,,,,,,,,,,,,,,
1,USA,"33,043,415","+58,616","589,186",+849,"25,639,773","6,814,456","9,774","99,347","1,771","445,521,945","1,339,488","332,606,157",2021-04-29
2,India,"18,754,925","+386,829","208,313","+3,501","15,369,362","3,177,250","8,944","13,481",150,"284,471,979","204,480","1,391,197,718",2021-04-29
3,Brazil,"14,592,886","+69,079","401,417","+3,074","13,091,714","1,099,755","8,318","68,253","1,877","43,538,104","203,633","213,807,144",2021-04-29
4,France,"5,592,390","+26,538","104,224",+306,"4,498,598","989,568","5,804","85,520","1,594","75,604,728","1,156,160","65,392,975",2021-04-29
5,Russia,"4,796,557","+9,284","109,731",+364,"4,419,540","267,286","2,300","32,856",752,"128,800,000","882,275","145,986,248",2021-04-29


In [4]:
def draw_file_excel(file_name, df_country):
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter',datetime_format = "YYYY-MM-DD")
    old_df_country = pd.read_excel(writer, sheet_name= "country", index_col=0)
    frames = [old_df_country, df_country]
    result = pd.concat(frames)
    result.to_excel(writer, sheet_name= "country")
    writer.save()

In [5]:
draw_file_excel("raw_data.xlsx", df_country)